In [ ]:
# para no usar GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf

2025-04-27 13:32:09.670164: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-27 13:32:09.670192: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-27 13:32:09.670963: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-27 13:32:09.675960: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-27 13:32:10.322910: W tensorflow/compiler/tf2

In [16]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# lista de acciones y precio de petroleo y oro
tickers = ['GOOG', 'AMZN', 'META', 'TSLA', 'AAPL', 'MSFT', 'NVDA', 'CL=F', 'GC=F']
data = yf.download(tickers, start='2010-01-01', end='2024-12-31', auto_adjust=False)['Adj Close']
data = data.ffill().bfill()

[*********************100%***********************]  9 of 9 completed


In [17]:
data

Ticker,AAPL,AMZN,CL=F,GC=F,GOOG,META,MSFT,NVDA,TSLA
Date,,,,,,,,,
2010-01-04,6.440329,6.695000,81.510002,1117.699951,15.536652,38.050671,23.254049,0.423883,1.592667
2010-01-05,6.451466,6.734500,81.769997,1118.099976,15.468232,38.050671,23.261568,0.430073,1.592667
2010-01-06,6.348847,6.612500,83.180000,1135.900024,15.078297,38.050671,23.118811,0.432824,1.592667
2010-01-07,6.337111,6.500000,82.660004,1133.099976,14.727282,38.050671,22.878386,0.424342,1.592667
2010-01-08,6.379240,6.676000,82.750000,1138.199951,14.923614,38.050671,23.036158,0.425259,1.592667
...,...,...,...,...,...,...,...,...,...
2024-12-23,254.989655,225.059998,69.239998,2612.300049,195.766968,599.316772,434.379028,139.657150,430.600006
2024-12-24,257.916443,229.050003,70.099998,2620.000000,197.345184,607.209778,438.450836,140.207108,462.279999
2024-12-26,258.735504,227.050003,69.620003,2638.800049,196.875717,602.813660,437.233276,139.917130,454.130005


In [18]:
def create_windows(data, window_size=101, max_samples=3000):
    series = []
    for i in range(0, len(data)-window_size+1, 1):
        window = data.iloc[i:i+window_size].T
        series.append(window.values)
        if len(series) == max_samples:
            break
    return np.array(series)

In [19]:
# escalamiento de datos para que estén entre 0 y 1
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)
data_scaled = pd.DataFrame(data_scaled, index=data.index, columns=data.columns)

series = create_windows(data_scaled)

X = series[:, :, :100]
Y = series[:, :, 100:]

print("Datos preparados:")
print("X shape:", X.shape)
print("Y shape:", Y.shape)


Datos preparados:
X shape: (3000, 9, 100)
Y shape: (3000, 9, 1)


In [20]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Asumimos que tienes: 
# X → (3000, 9, 100), Y → (3000, 9, 1)

# Transponer para que sea (batch, time, features)
X_tf = np.transpose(X, (0, 2, 1))  # (3000, 100, 9)
Y_tf = Y.squeeze(-1)  # (3000, 9)

# Dividir en entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X_tf, Y_tf, test_size=0.2, random_state=42)


In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout

model = Sequential([
    GRU(128, return_sequences=True, input_shape=(100, 9)),
    Dropout(0.2),
    GRU(64),
    Dense(9)
])

model.compile(optimizer='adam', loss='mse')
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_4 (GRU)                 (None, 100, 128)          53376     
                                                                 
 dropout_1 (Dropout)         (None, 100, 128)          0         
                                                                 
 gru_5 (GRU)                 (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 9)                 585       
                                                                 
Total params: 91209 (356.29 KB)
Trainable params: 91209 (356.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
history = model.fit(
    X_train, Y_train,
    validation_data=(X_test, Y_test),
    epochs=10,
    batch_size=64
)

Epoch 1/10
38/38 [==============================] - 9s 152ms/step - loss: 0.0133 - val_loss: 0.0015
Epoch 2/10
38/38 [==============================] - 5s 128ms/step - loss: 0.0014 - val_loss: 5.2579e-04
Epoch 3/10
38/38 [==============================] - 5s 134ms/step - loss: 8.9214e-04 - val_loss: 3.8668e-04
Epoch 4/10
38/38 [==============================] - 5s 124ms/step - loss: 6.8240e-04 - val_loss: 3.1160e-04
Epoch 5/10
38/38 [==============================] - 5s 137ms/step - loss: 5.9964e-04 - val_loss: 2.5842e-04
Epoch 6/10
38/38 [==============================] - 5s 122ms/step - loss: 5.0158e-04 - val_loss: 2.2377e-04
Epoch 7/10
38/38 [==============================] - 5s 133ms/step - loss: 4.4699e-04 - val_loss: 2.4994e-04
Epoch 8/10
38/38 [==============================] - 5s 122ms/step - loss: 4.0259e-04 - val_loss: 1.8902e-04
Epoch 9/10
38/38 [==============================] - 5s 137ms/step - loss: 3.7966e-04 - val_loss: 1.8211e-04
Epoch 10/10
38/38 [=====================

In [26]:
loss = model.evaluate(X_test, Y_test)
print(f"MSE en test: {loss:.6f}")

19/19 [==============================] - 0s 15ms/step - loss: 1.7968e-04
MSE en test: 0.000180


In [27]:
baseline_pred = X_test[:, -1, :]
baseline_loss = tf.reduce_mean(tf.square(baseline_pred - Y_test)).numpy()

print(f"Loss predictor trivial: {baseline_loss:.6f}")
print(f"Loss GRU entrenado: {loss:.6f}")

Loss predictor trivial: 0.000040
Loss GRU entrenado: 0.000180
